# logit-регрессия: LR-тест 

In [1]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from scipy.stats import chi2 # критические значения chi2

# Не показывать FutureWarnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# импорт датасета
df = pd.read_csv('https://raw.githubusercontent.com/artamonoff/econometrica/main/econometrica2/data-csv/loanapp.csv', na_values=(' ', '', '  '))
# импорт данных из локального файла
# df = pd.read_csv('loanapp.csv')
df

,occ,loanamt,action,msa,suffolk,appinc,typur,unit,married,dep,...,approve,mortno,mortperf,mortlat1,mortlat2,chist,multi,loanprc,thick,white
0,1,89,1,1120,0,72,0,1.0,0.0,0.0,...,1,0,1,0,0,1,0.0,0.754237,0.0,1
1,1,128,3,1120,0,74,0,1.0,1.0,1.0,...,0,0,1,0,0,1,0.0,0.800000,1.0,1
2,1,128,1,1120,0,84,3,1.0,0.0,0.0,...,1,0,1,0,0,1,0.0,0.895105,1.0,1
3,1,66,1,1120,0,36,0,1.0,1.0,0.0,...,1,0,1,0,0,0,0.0,0.600000,0.0,1
4,1,120,1,1120,0,59,8,1.0,1.0,0.0,...,1,0,1,0,0,1,0.0,0.895522,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1984,1,158,1,1120,0,96,0,1.0,1.0,0.0,...,1,0,1,0,0,1,0.0,0.897727,0.0,1
1985,1,35,1,1120,0,169,1,1.0,1.0,0.0,...,1,1,0,0,0,1,0.0,0.111111,0.0,1
1986,2,225,1,1120,0,49,0,2.0,1.0,0.0,...,1,0,1,0,0,1,1.0,1.000000,0.0,1
1987,1,98,1,1120,1,110,1,1.0,0.0,0.0,...,1,1,0,0,0,1,0.0,0.455814,0.0,1


# LR-тест: значимость регресии

## Спецификация и подгонка модели
Для датасета `loanapp`
рассморим logit-регрессию **approve на appinc, appinc^2, mortno, unem, dep, male, married, yjob, self**

In [3]:
mod = smf.logit(formula='approve~appinc+I(appinc**2)+mortno+unem+dep+male+married+yjob+self', data=df) # спецификация модели
res = mod.fit() # подгонка модели
res.summary() # отчет

Optimization terminated successfully.
         Current function value: 0.362116
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                approve   No. Observations:                 1971
Model:                          Logit   Df Residuals:                     1961
Method:                           MLE   Df Model:                            9
Date:                Thu, 23 Nov 2023   Pseudo R-squ.:                 0.03286
Time:                        12:58:34   Log-Likelihood:                -713.73
converged:                       True   LL-Null:                       -737.98
Covariance Type:            nonrobust   LLR p-value:                 2.063e-07
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          1.6806      0.228      7.363      0.000       1.233       2.128
appinc             0.0042      0.002      1.724      0.085      -0.001       0.009
I(appinc ** 2) -7.338e-06    3.3e-06     -2.226      0.026   -1.38e-05   -8.76e-07
mortno             0.7072      0.175      4.042      0.000       0.364       1.050
unem              -0.0498      0.030     -1.685      0.092      -0.108       0.008
dep               -0.1545      0.065     -2.371      0.018      -0.282      -0.027
male              -0.0207      0.188     -0.110      0.912      -0.388       0.347
married            0.3977      0.163      2.438      0.015       0.078       0.718
yjob              -0.0101      0.065     -0.156      0.876      -0.137       0.117
self              -0.3593      0.200     -1.801      0.072      -0.750       0.032
==================================================================================
"""

In [4]:
# Число наблюдений, по которым была оценена модель
res.nobs

1971

In [5]:
# логарифм функции правдоподобия для модели
res.llf.round(3)

-713.731

In [6]:
# логарфим функции правдоподобия для регрессии без объясняющих переменных (только на константу)
res.llnull.round(3)

-737.979

## Гипотеза
Тестируем значимость регрессии, т.е. гипотезу $$H_0:\beta_{appinc}=\beta_{appinc^2}=\beta_{mortno}=\beta_{unem}=\beta_{dep}=\beta_{male}=\beta_{married}=\beta_{yjob}=\beta_{self}=0.$$

## Уровень значимости
Пусть уровень значимости $\alpha=5\%=0.05$

In [7]:
# Тестовая статистика LR-теста и её P-значение с округленим
res.llr.round(3), res.llr_pvalue.round(3)

(48.496, 0.0)

### Критическое значение $\chi_{df=k}^2(\alpha)$

In [8]:
sign_level = 0.1 # уровень значимости
chi2.ppf(q=1-sign_level, df=res.df_model).round(3) 

14.684

### Значима ли регрессия? 

`Способ 1`

---
Отвергаем нулевую гипотезу при $LR_{stat}>\chi_{df=k}^2(\alpha)$, `регрессия значима`

Не отвергаем нулевую гипотезу при $LR_{stat}<\chi_{df=k}^2(\alpha)$, `регрессия незначима`

---

`Способ 2`

---
Отвергаем нулевую гипотезу при $P<\alpha$, `регрессия значима`

Не отвергаем нулевую гипотезу при $P>\alpha$, `регрессия незначима`

---

### **<span style="color:#59afe1">Вывод о значимости регрессии</span>**, используя критическое значение

`регрессия незначима` $(LR_{stat}<\chi_{df=k}^2(\alpha))$

### **<span style="color:#59afe1">Вывод о значимости регрессии</span>**, используя  $P$-значение

`регрессия незначима` ($P>\alpha$)

# LR-тест: совместная значимость

## Гипотеза
Тестируем значимость влияния дохода, т.е. $$H_0:\beta_{appinc}=\beta_{appinc^2}=0.$$

## Уровень значимости
Пусть уровень значимости $\alpha=1\%=0.01$

In [9]:
restricted = smf.logit(formula='approve~mortno+unem+dep+male+married+yjob+self', data=df).fit() # спецификация и подгонка модели

Optimization terminated successfully.
         Current function value: 0.363993
         Iterations 7


In [10]:
res.compare_lr_test(restricted)

AttributeError: 'LogitResults' object has no attribute 'compare_lr_test'

## Формула для расчета $LR$ статистики

In [16]:
# вычислим тестовую статистику LR-теста и её P-значение по формуле
llf_full = res.llf
llf_restr = restricted.llf
df_full = res.df_resid 
df_restr = restricted.df_resid 
lr_df = (df_restr - df_full)
lr_stat = -2*(llf_restr - llf_full)
lr_pvalue = chi2.sf(lrstat, df=lr_df)
lr_stat, lr_pvalue

(7.399458185385811, 0.024730225161638596)

In [11]:
# Тестовая статистика LR-теста и её P-значение с округленим
res.wald_test('appinc=I(appinc ** 2)=0')

<class 'statsmodels.stats.contrast.ContrastResults'>
<Wald test (chi2): statistic=[[7.1202905]], p-value=0.028434694253215525, df_denom=2>

### Критическое значение $\chi_{df=df\_denom}^2(\alpha)$

In [ ]:
sign_level = 0.01 # уровень значимости
chi2.ppf(q=1-sign_level, df=2).round(3) 

9.21

### Значимо ли влияние дохода?

`влияние дохода незначимо` $(LR_{stat}<\chi_{df=df\_denom}^2(\alpha))$